# <center>Organizaci&oacute;n de Datos</center>
#### <center>C&aacute;tedra Ing. Rodriguez, Juan Manuel </center>

## <center>Trabajo Práctico 1:</center>
### <center>  Propiedades en Venta</center>
### <center>  Grupo 18</center>


#### **Integrantes:**

* Analía Acevedo
* Santiago Daniel Marczewski Neudeck
* Lucia Belén Napoli
* Brayan Ricaldi

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import sklearn.preprocessing as skp
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import (
    KBinsDiscretizer,
    LabelEncoder,
    MinMaxScaler,
    Normalizer,
    OneHotEncoder,
    OrdinalEncoder,
    PowerTransformer,
    RobustScaler,
    StandardScaler,
)

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
from preprocessing import preprocesamiento

from sklearn.model_selection import RandomizedSearchCV
from joblib import dump, load

pd.options.display.max_columns = None

Descargo los datos para el modelo 1

In [2]:
_x_train_model1 = pd.read_csv('./DataSet/x_train')
_x_train_model1.drop(columns=['Unnamed: 0'],inplace=True)
_y_train_model1 = pd.read_csv('./DataSet/y_train')
_y_train_model1.drop(columns=['Unnamed: 0'],inplace=True)
_x_test_model1 = pd.read_csv('./DataSet/x_test')
_x_test_model1.drop(columns=['Unnamed: 0'],inplace=True)
_y_test_model1 = pd.read_csv('./DataSet/y_test')
_y_test_model1.drop(columns=['Unnamed: 0'],inplace=True)
_y_train_model1.rename(columns={'0':'pxm2'}, inplace=True)
_y_test_model1.rename(columns={'0':'pxm2'}, inplace=True)

Descargo los datos para el modelo 2(ZSCORE-PCA)

In [3]:
_x_train_model2 = pd.read_csv('./DataSet-Modificados/ZSCORE-PCA/x_train')
_x_train_model2.drop(columns=['Unnamed: 0'],inplace=True)
_y_train_model2 = pd.read_csv('./DataSet-Modificados/ZSCORE-PCA/y_train')
_y_train_model2.drop(columns=['Unnamed: 0'],inplace=True)
_x_test_model2 = pd.read_csv('./DataSet-Modificados/ZSCORE-PCA/x_test')
_x_test_model2.drop(columns=['Unnamed: 0'],inplace=True)
_y_test_model2 = pd.read_csv('./DataSet-Modificados/ZSCORE-PCA/y_test')
_y_test_model2.drop(columns=['Unnamed: 0'],inplace=True)
_y_train_model2.rename(columns={'0':'pxm2'}, inplace=True)
_y_test_model2.rename(columns={'0':'pxm2'}, inplace=True)

### **4. Regresión**

En esta etapa se busca predecir el precio de la propiedad utilizando dos modelos diferentes.
Para todos los modelos se pide:

* Realizar las tareas de ingeniería de características necesarias para trabajar con
cada algoritmo (encoding, normalización, etc)
* Todos los modelos deben ser entrenados también con el dataset generado en el
punto **1 d)**

## Modelo 1: KNN

a. Construir un modelo [KNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor) para regresión y optimizar sus hiperparámetros
mediante *k-fold Cross Validation* para obtener la mejor performance .¿Cuántos
*folds* utilizaron? ¿Qué métrica consideran adecuada para buscar los parámetros?

b. Evaluar la performance del modelo en el conjunto de evaluación, explicar todas
las métricas. Comparar con la performance de entrenamiento.

### 1er modelo

In [4]:
#Ejecutare mis modelos con una copia
x_train = _x_train_model1.copy()
y_train = _y_train_model1.copy()
x_test = _x_test_model1.copy()
y_test = _y_test_model1.copy()

Normalizo con min-max

In [5]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

Busco los mejores parametros para KNN con respecto Root Mean Squared Error.

In [6]:
#Grilla de Parámetros
params_grid={ 'n_neighbors':range(5,25), 
              'weights':['distance','uniform'],
              'algorithm':['ball_tree', 'kd_tree', 'brute'],
              'metric':['euclidean','manhattan','chebyshev']
             }

#Clasificador KNN
knn=KNeighborsRegressor()

#Random Search con 10 Folds y 10 iteraciones
rand_knn_model1 = RandomizedSearchCV(knn, params_grid, scoring='neg_root_mean_squared_error', cv=5, n_iter=10, random_state=5)

rand_knn_model1.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsRegressor(),
                   param_distributions={'algorithm': ['ball_tree', 'kd_tree',
                                                      'brute'],
                                        'metric': ['euclidean', 'manhattan',
                                                   'chebyshev'],
                                        'n_neighbors': range(5, 25),
                                        'weights': ['distance', 'uniform']},
                   random_state=5, scoring='neg_root_mean_squared_error')

In [7]:
rand_knn_model1.best_estimator_

KNeighborsRegressor(algorithm='brute', metric='chebyshev', n_neighbors=23)

In [8]:
print(f"Observamos que los parametros que performaron mejor son: {rand_knn_model1.best_estimator_}")

Observamos que los parametros que performaron mejor son: KNeighborsRegressor(algorithm='brute', metric='chebyshev', n_neighbors=23)


In [9]:
print(f"Y el error mas bajo fue: {rand_knn_model1.best_score_}")

Y el error mas bajo fue: -1223.8241565096353


Ahora veamos como nos anda en test

In [10]:
#Mejor estimador
best_knn=rand_knn_model1.best_estimator_

#Predicción con mejor estimador
y_pred=best_knn.predict(x_test)

#RootMean Squared Error
mean_squared_error(y_test, y_pred, squared=False)

1217.401032628558

In [29]:
rand_knn_model1.best_estimator_

KNeighborsRegressor(algorithm='brute', metric='chebyshev', n_neighbors=23)

Lo guardamos

In [33]:
dump(rand_knn_model1.best_estimator_, './Modelos/KNN-model1.joblib') 

['./Modelos/KNN-model1.joblib']

No se observa una gran diferencia entre el score del train con el del test.
Al parecer el modelo no esta haciendo overfitting y creemos que es un buen resultado.

### 2do modelo

In [11]:
#Ejecutare_estimator_are mis modelos con una copia
x_train = _x_train_model2.copy()
y_train = _y_train_model2.copy()
x_test = _x_test_model2.copy()
y_test = _y_test_model2.copy()

Busco los mejores parametros para KNN con respecto Root Mean Squared Error.

In [12]:
#Grilla de Parámetros
params_grid={ 'n_neighbors':range(5,25), 
              'weights':['distance','uniform'],
              'algorithm':['ball_tree', 'kd_tree', 'brute'],
              'metric':['euclidean','manhattan','chebyshev']
             }

#Clasificador KNN
knn=KNeighborsRegressor()

#Random Search con 10 Folds y 10 iteraciones
rand_knn_model2 = RandomizedSearchCV(knn, params_grid, scoring='neg_root_mean_squared_error', cv=5, n_iter=10, random_state=5)

rand_knn_model2.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsRegressor(),
                   param_distributions={'algorithm': ['ball_tree', 'kd_tree',
                                                      'brute'],
                                        'metric': ['euclidean', 'manhattan',
                                                   'chebyshev'],
                                        'n_neighbors': range(5, 25),
                                        'weights': ['distance', 'uniform']},
                   random_state=5, scoring='neg_root_mean_squared_error')

In [13]:
print(f"Observamos que los parametros que performaron mejor son: {rand_knn_model2.best_estimator_}")

Observamos que los parametros que performaron mejor son: KNeighborsRegressor(algorithm='brute', metric='chebyshev', n_neighbors=23)


In [14]:
print(f"Y el error mas bajo fue: {rand_knn_model2.best_score_}")

Y el error mas bajo fue: -1239.339511601893


Ahora veamos como nos anda en test

In [15]:
#Mejor estimador
best_knn=rand_knn_model2.best_estimator_

#Predicción con mejor estimador
y_pred=best_knn.predict(x_test)

#RootMean Squared Error
mean_squared_error(y_test, y_pred, squared=False)

718.1341916561961

Se observa una gran diferencia entre el score del train con el del test, es casi la mitad.

Lo guardamos

In [34]:
dump(rand_knn_model2.best_estimator_, './Modelos/KNN-model2.joblib') 

['./Modelos/KNN-model2.joblib']

## Modelo 2: XGBoost

a. Construir un modelo **XGBoost** y optimizar sus hiperparámetros mediante *k-fold
Cross Validation* para obtener la mejor performance. ¿Cuántos *folds* utilizaron?
¿Qué métrica consideran adecuada para buscar los parámetros?

b. Evaluar la performance del modelo en el conjunto de evaluación, explicar todas
las métricas. Comparar con la performance de entrenamiento.

In [16]:
import xgboost as xgb

### 1er modelo

In [17]:
#Ejecutare mis modelos con una copia
x_train = _x_train_model1.copy()
y_train = _y_train_model1.copy()
x_test = _x_test_model1.copy()
y_test = _y_test_model1.copy()

Normalizo con ZScore

In [18]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [19]:
#Grilla de Parámetros
params_grid={
    'n_estimators': [100, 500, 900, 1100, 1500],
    'max_depth': [2, 3, 5, 10, 15],
    'learning_rate': [0.05, 0.1, 0.15, 0.20],
    'min_child_weight': [1, 2, 3, 4]
    }

#Clasificador XGBRegressor
xgbr = xgb.XGBRegressor(random_state = 0)

#Random Search con 5 Folds y 8 iteraciones
rand_xgboost_model1 = RandomizedSearchCV(xgbr, params_grid, scoring='neg_root_mean_squared_error', cv=5, n_iter=8, random_state=5)

rand_xgboost_model1.fit(x_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          enable_categorical=False, gamma=None,
                                          gpu_id=None, importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=...
                                          predictor=None, random_state=0,
                                          reg_alpha=None, reg_lambda=None,
                                          

In [20]:
print(f"Observamos que los parametros que performaron mejor son: {rand_xgboost_model1.best_estimator_}")

Observamos que los parametros que performaron mejor son: XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=5, min_child_weight=3, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)


In [21]:
print(f"Y el error mas bajo fue: {rand_xgboost_model1.best_score_}")

Y el error mas bajo fue: -1233.6146529699324


Ahora veamos como nos anda en test

In [22]:
#Mejor estimador
best_xgbr=rand_xgboost_model1.best_estimator_

#Predicción con mejor estimador
y_pred=best_xgbr.predict(x_test)

#RootMean Squared Error
mean_squared_error(y_test, y_pred, squared=False)

906.6026978535414

Observamos un score bajo en el test, al parecer el modelo funciona correctamente con nuevos datos

Lo guardamos

In [35]:
dump(rand_xgboost_model1.best_estimator_, './Modelos/XGBOOST-model1.joblib') 

['./Modelos/XGBOOST-model1.joblib']

### 2do modelo

In [23]:
#Ejecutare_estimator_are mis modelos con una copia
x_train = _x_train_model2.copy()
y_train = _y_train_model2.copy()
x_test = _x_test_model2.copy()
y_test = _y_test_model2.copy()

Busco los mejores parametros para XGBOOST con respecto Root Mean Squared Error.

In [24]:
#Grilla de Parámetros
params_grid={
    'n_estimators': [100, 500, 900, 1100, 1500],
    'max_depth': [2, 3, 5, 10, 15],
    'learning_rate': [0.05, 0.1, 0.15, 0.20],
    'min_child_weight': [1, 2, 3, 4]
    }

#Clasificador XGBRegressor
xgbr = xgb.XGBRegressor(random_state = 0)

#Random Search con 5 Folds y 8 iteraciones
rand_xgboost_model2 = RandomizedSearchCV(xgbr, params_grid, scoring='neg_root_mean_squared_error', cv=5, n_iter=8, random_state=5)

rand_xgboost_model2.fit(x_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          enable_categorical=False, gamma=None,
                                          gpu_id=None, importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=...
                                          predictor=None, random_state=0,
                                          reg_alpha=None, reg_lambda=None,
                                          

In [25]:
print(f"Observamos que los parametros que performaron mejor son: {rand_xgboost_model2.best_estimator_}")

Observamos que los parametros que performaron mejor son: XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=5, min_child_weight=3, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)


In [26]:
print(f"Y el error mas bajo fue: {rand_xgboost_model2.best_score_}")

Y el error mas bajo fue: -1239.3398821556414


Ahora veamos como nos anda en test

In [27]:
#Mejor estimador
best_xgbr=rand_xgboost_model2.best_estimator_

#Predicción con mejor estimador
y_pred=best_xgbr.predict(x_test)

#RootMean Squared Error
mean_squared_error(y_test, y_pred, squared=False)

641.5620151352072

Lo guardamos

In [36]:
dump(rand_xgboost_model2.best_estimator_, './Modelos/XGBOOST-model2.joblib') 

['./Modelos/XGBOOST-model2.joblib']

## Modelo 3: Arbol de Regresion

In [4]:
from sklearn.tree import DecisionTreeRegressor

### 1er modelo

In [5]:
#Ejecutare mis modelos con una copia
x_train = _x_train_model1.copy()
y_train = _y_train_model1.copy()
x_test = _x_test_model1.copy()
y_test = _y_test_model1.copy()

Normalizo con min-max

In [6]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

Busco los mejores parametros para mi Arbol de Regresion con respecto Root Mean Squared Error.

In [8]:
#Grilla de Parámetros
params_grid = { "criterion" : ["squared_error", "friedman_mse", "absolute_error"], 
               "min_samples_leaf" : [5, 10], 
               "min_samples_split" : [2, 4, 10, 12, 16], 
               "splitter": ['random','best'] }

#Clasificador Arbol
tree = DecisionTreeRegressor(random_state = 5)

#Random Search con 5 Folds y 10 iteraciones
rand_tree_model1 = RandomizedSearchCV(tree, params_grid, scoring='neg_root_mean_squared_error', cv=5, n_iter=10, random_state=5)

rand_tree_model1.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=5),
                   param_distributions={'criterion': ['squared_error',
                                                      'friedman_mse',
                                                      'absolute_error'],
                                        'min_samples_leaf': [5, 10],
                                        'min_samples_split': [2, 4, 10, 12, 16],
                                        'splitter': ['random', 'best']},
                   random_state=5, scoring='neg_root_mean_squared_error')

In [9]:
print(f"Observamos que los parametros que performaron mejor son: {rand_tree_model1.best_estimator_}")

Observamos que los parametros que performaron mejor son: DecisionTreeRegressor(criterion='absolute_error', min_samples_leaf=5,
                      min_samples_split=4, random_state=5)


In [10]:
print(f"Y el error mas bajo fue: {rand_tree_model1.best_score_}")

Y el error mas bajo fue: -1171.865033492837


Ahora veamos como nos anda en test

In [11]:
#Mejor estimador
best_svr=rand_tree_model1.best_estimator_

#Predicción con mejor estimador
y_pred=best_svr.predict(x_test)

#RootMean Squared Error
mean_squared_error(y_test, y_pred, squared=False)

1077.7757830732962

Lo guardamos

In [17]:
dump(rand_tree_model1.best_estimator_, './Modelos/DecisionTreeRegressor-model1.joblib') 

['./Modelos/DecisionTreeRegressor-model1.joblib']

### 2do modelo

In [12]:
#Ejecutare_estimator_are mis modelos con una copia
x_train = _x_train_model2.copy()
y_train = _y_train_model2.copy()
x_test = _x_test_model2.copy()
y_test = _y_test_model2.copy()

Busco los mejores parametros para mi Arbol de Regresion con respecto Root Mean Squared Error.

In [13]:
#Grilla de Parámetros
params_grid = { "criterion" : ["squared_error", "friedman_mse", "absolute_error"], 
               "min_samples_leaf" : [5, 10], 
               "min_samples_split" : [2, 4, 10, 12, 16], 
               "splitter": ['random','best'] }

#Clasificador Arbol
tree = DecisionTreeRegressor(random_state = 5)

#Random Search con 5 Folds y 10 iteraciones
rand_tree_model2 = RandomizedSearchCV(tree, params_grid, scoring='neg_root_mean_squared_error', cv=5, n_iter=10, random_state=5)

rand_tree_model2.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=5),
                   param_distributions={'criterion': ['squared_error',
                                                      'friedman_mse',
                                                      'absolute_error'],
                                        'min_samples_leaf': [5, 10],
                                        'min_samples_split': [2, 4, 10, 12, 16],
                                        'splitter': ['random', 'best']},
                   random_state=5, scoring='neg_root_mean_squared_error')

In [14]:
print(f"Observamos que los parametros que performaron mejor son: {rand_tree_model2.best_estimator_}")

Observamos que los parametros que performaron mejor son: DecisionTreeRegressor(criterion='absolute_error', min_samples_leaf=10,
                      min_samples_split=4, random_state=5, splitter='random')


In [15]:
print(f"Y el error mas bajo fue: {rand_tree_model2.best_score_}")

Y el error mas bajo fue: -1195.969554193854


Ahora veamos como nos anda en test

In [16]:
#Mejor estimador
best_svr=rand_tree_model2.best_estimator_

#Predicción con mejor estimador
y_pred=best_svr.predict(x_test)

#RootMean Squared Error
mean_squared_error(y_test, y_pred, squared=False)

655.5528626402848

Lo guardamos

In [18]:
dump(rand_tree_model2.best_estimator_, './Modelos/DecisionTreeRegressor-model2.joblib') 

['./Modelos/DecisionTreeRegressor-model2.joblib']

## Resumen de RMSE en test:
   + KNN - Model 1: 1217.401032628558
   + KNN - Model 2: 718.1341916561961
   + XGBoost - Model 1: 906.6026978535414
   + XGBoost - Model 2: 641.5620151352072
   + Arbol de Regresion - Model 1: 1077.7757830732962
   + Arbol de Regresion - Model 2: 655.5528626402848

Observamos que los modelos entrenados con el dataSet reducido nos dan mejores resultados.

**¿Qué modelo elegirían para predecir el precio de venta de las propiedades?**<br>
Elegiría el modelo 2 de árbol de decisión, porque es el más intuitivo y fácil de entender, además es el 2do con mejor score.